# 3-Manifolds and VOA characters
## Example section companion notebook

Non pseudospherical examples are the most general types of Seifert manifolds. For these, the lattice dilation factor $D$ is greater than one and less-than-or-equal-to the determinant of the plumbing matrix. In this example notebook, we replicate the computations done in writing the fourth paragraph of section 6.1 of arXiv:2201.04640, while at the same time demonstrating the capabilities and main functions of pySeifert: a Sage toolkit for the computation of $\hat{Z}$ invariants for Seifert manifolds and related q-series invariants.

We start by loading pySeifert and defining the Seifert manifold

In [1]:
# Load pySeifert
load("../../pySeifert.sage")
S = Seifert([0,-1,3,2,7,-1,2]);S

Seifert manifold with 3 exceptional fibers.
Seifert data:
[0, -1, 3, 2, 7, -1, 2]
Plumbing Matrix:
[ 0  1  1  0  1]
[ 1  3  0  0  0]
[ 1  0 -4  1  0]
[ 0  0  1 -2  0]
[ 1  0  0  0  2]
D: 23, m: 42, det(M): -23

Because $D>1$, $S$ is not a pseudospherical Seifert manifold.

Boundary conditions are labeled by:

In [2]:
B = S.boundary_conditions(["A",2], basis = "weight"); B

[[(-1, -1), (1, 1), (0, 0), (1, 1), (1, 1)],
 [(1, -2), (3, 0), (-4, 2), (1, 1), (3, 0)],
 [(1, -2), (3, 0), (-2, 1), (1, 1), (3, 0)],
 [(1, -2), (3, 0), (0, 0), (-1, 2), (5, -1)],
 [(1, -2), (5, -1), (0, 0), (-1, 2), (3, 0)],
 [(1, -2), (7, -2), (0, 0), (1, 1), (3, 0)],
 [(3, -3), (3, 0), (-6, 3), (1, 1), (5, -1)],
 [(3, -3), (5, -1), (-6, 3), (1, 1), (3, 0)],
 [(3, -3), (5, -1), (-4, 2), (-1, 2), (5, -1)],
 [(3, -3), (5, -1), (-2, 1), (-1, 2), (5, -1)],
 [(3, -3), (7, -2), (-4, 2), (-1, 2), (3, 0)],
 [(3, -3), (7, -2), (-2, 1), (1, 1), (5, -1)]]

$\mathcal B$ contains the trivial $\vec{\underline{b}}_0$ and a non trivial conditions $\vec {\underline{b}}_i$, $i=1,10$. For $\vec{\underline{b}}_0$ the $\vec{s}$ values are:

In [3]:
b0 = B[0]
lVals,sVals,kVals = list(zip(*S.s_values(["A",2],b0, basis = "weight")))
len(sVals),list(zip(lVals,sVals,kVals))[:3]

(216,
 [(-1, (-29, -29), (2, 2)),
  (1, (13, -50), (26, -10)),
  (1, (-50, 13), (-10, 26))])

Because the inverse of the plumbing matrix contains non-integer entries, not all $\vec s $ values are admissible. Here, only $72$ values are admissible.

For each $\vec s$ we can compute the associated $\hat Z$ invariant integrand  $\tilde{\chi}_{\hat{w};\vec{\underline{b}}}$. For example, choosing $\hat w = (1,1,1)$ leads to:

In [4]:
wh = [identity_matrix(2) if d == 1 else matrix(2) for d in S.deg]
rho = vector([1,1])
s = -S.d*S.A*matrix([wi*rho for wi in wh]);s
chi_tilde = S.chi_tilde(["A",2],wh, b0, 100, basis="weight"); chi_tilde

(z0^2*z1^2 - z0^4/z1^2 - z1^4/z0^2 + z0^2/z1^4 + z1^2/z0^4 - 1/(z0^2*z1^2))*q^(298/23)/(z0*z1 - z0^2/z1 - z1^2/z0 + z0/z1^2 + z1/z0^2 - 1/(z0*z1))

For each $\vec s$ we can also compute the triplet character. Choosing the same $\hat{w}$ we get:

In [5]:
mu = 1/sqrt(S.m)*(rho-s)
trip_ch = triplet_character(["A",2],vector([0,0]),mu,S.m,3,100,basis="weight"); trip_ch

-2*q^(3823/42) - (z0^2*z1^2 - z0^4/z1^2 - z1^4/z0^2 + z0^2/z1^4 + z1^2/z0^4 - 1/(z0^2*z1^2))*q^(3025/42)/(z0*z1 - z0^2/z1 - z1^2/z0 + z0/z1^2 + z1/z0^2 - 1/(z0*z1)) + 2*q^(1387/42) - q^(169/42)

By summing over boundary conditions, properly dilating $\tau$ and multiplying by $q^\delta$ we can relate the $\hat{Z}$ integrand to the triplet character.

In [6]:
chi_tilde_sum = 0
cart = cartan_matrix(["A",2])
for db in itertools.product(range(S.d),repeat=2):
    db = cart*vector(db)
    chi_tilde_sum += S.chi_tilde(["A",2],wh, [b0[0]+db]+b0[1:], 100, basis="weight")

dil_chi_tilde = 0
delta = S.delta(["A",2])
for pref, exp in chi_tilde_sum.coefficients(q):
    dil_chi_tilde += pref * q^(S.d*(exp-delta))

expand(dil_chi_tilde-trip_ch).series(q,100).truncate()

0